# Parse the DrugBank XML and extract TSVs

Run using Python 3 to avoid a non-ascii character error when writing to file with the csv module.

In [1]:
import os
import csv
import gzip
import collections
import re
import io
import json
import xml.etree.ElementTree as ET

import requests
import pandas

In [2]:
# 원본
# xml_path = os.path.join('download', 'drugbank.xml.gz')
# with gzip.open(xml_path) as xml_file:
#     tree = ET.parse(xml_file)
# root = tree.getroot()

In [3]:
# drugbank full database - 2021 year
xml_path='D:/WorkSpace/master_thesis/Drug_Bank/download_drugbank/full_database.xml.gz'
with gzip.open(xml_path) as xml_file:
    tree = ET.parse(xml_file)
root = tree.getroot()

In [146]:
ns = '{http://www.drugbank.ca}'
inchikey_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChIKey']/{ns}value"
inchi_template = "{ns}calculated-properties/{ns}property[{ns}kind='InChI']/{ns}value"

rows = list()
for i, drug in enumerate(root):
    row = collections.OrderedDict()
    assert drug.tag == ns + 'drug'
    row['type'] = drug.get('type')
    row['drugbank_id'] = drug.findtext(ns + "drugbank-id[@primary='true']")
    row['name'] = drug.findtext(ns + "name")
    row['description'] = drug.findtext(ns + "description")
    row['groups'] = [group.text for group in
        drug.findall("{ns}groups/{ns}group".format(ns = ns))]
    row['atc_codes'] = [code.get('code') for code in
        drug.findall("{ns}atc-codes/{ns}atc-code".format(ns = ns))]
    row['targets'] = [target.text for target in
        drug.findall("{ns}targets/{ns}target/{ns}name".format(ns = ns))]
    row['categories'] = [x.findtext(ns + 'category') for x in
        drug.findall("{ns}categories/{ns}category".format(ns = ns))]
    row['inchi'] = drug.findtext(inchi_template.format(ns = ns))
    row['inchikey'] = drug.findtext(inchikey_template.format(ns = ns))
    
  
    
    # Add drug aliases
    aliases = {
        elem.text for elem in 
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}synonyms/{ns}synonym[@language='English']".format(ns = ns)) +
        drug.findall("{ns}international-brands/{ns}international-brand".format(ns = ns)) +
        drug.findall("{ns}products/{ns}product/{ns}name".format(ns = ns))

    }
    aliases.add(row['name'])
    row['aliases'] = sorted(aliases)

    rows.append(row)

In [147]:
alias_dict = {row['drugbank_id']: row['aliases'] for row in rows}
with open('./data/aliases.json', 'w') as fp:
    json.dump(alias_dict, fp, indent=2, sort_keys=True)

In [148]:
def collapse_list_values(row):
    for key, value in row.items():
        if isinstance(value, list):
            row[key] = '|'.join(value)
    return row

rows = list(map(collapse_list_values, rows))

In [149]:
columns = ['drugbank_id', 'name', 'type', 'groups', 'atc_codes', 'targets', 'categories', 'inchikey', 'inchi', 'description']
drugbank_df = pandas.DataFrame.from_dict(rows)[columns]
drugbank_df.head()

,drugbank_id,name,type,groups,atc_codes,targets,categories,inchikey,inchi,description
0,DB00001,Lepirudin,biotech,approved,B01AE02,Prothrombin,"Amino Acids, Peptides, and Proteins|Anticoagul...",None,None,Lepirudin is identical to natural hirudin exce...
1,DB00002,Cetuximab,biotech,approved,L01XC06,Epidermal growth factor receptor|Low affinity ...,"Amino Acids, Peptides, and Proteins|Antibodies...",None,None,Cetuximab is an epidermal growth factor recept...
2,DB00003,Dornase alfa,biotech,approved,R05CB13,DNA,"Amino Acids, Peptides, and Proteins|Cough and ...",None,None,Dornase alfa is a biosynthetic form of human d...
3,DB00004,Denileukin diftitox,biotech,approved|investigational,L01XX29,Interleukin-2 receptor subunit alpha|Interleuk...,"ADP Ribose Transferases|Amino Acids, Peptides,...",None,None,A recombinant DNA-derived cytotoxic protein co...
4,DB00005,Etanercept,biotech,approved|investigational,L04AB01,Tumor necrosis factor|High affinity immunoglob...,"Agents reducing cytokine levels|Amino Acids, P...",None,None,Dimeric fusion protein consisting of the extra...


In [150]:
drugbank_slim_df = drugbank_df[
    drugbank_df.groups.map(lambda x: 'approved' in x) &
    drugbank_df.inchi.map(lambda x: x is not None) &
    drugbank_df.type.map(lambda x: x == 'small molecule')
]
drugbank_slim_df.head()

,drugbank_id,name,type,groups,atc_codes,targets,categories,inchikey,inchi,description
5,DB00006,Bivalirudin,small molecule,approved|investigational,B01AE06,Prothrombin,"Amino Acids, Peptides, and Proteins|Anticoagul...",OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,Bivalirudin is a synthetic 20 residue peptide ...
6,DB00007,Leuprolide,small molecule,approved|investigational,L02AE51|L02AE02,Gonadotropin-releasing hormone receptor,Adrenal Cortex Hormones|Agents Causing Muscle ...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,Leuprolide is a synthetic 9-residue peptide an...
13,DB00014,Goserelin,small molecule,approved,L02AE03,Lutropin-choriogonadotropic hormone receptor|G...,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,"Goserelin is a synthetic hormone. In men, it s..."
25,DB00027,Gramicidin D,small molecule,approved,R02AB30,,"Amino Acids, Peptides, and Proteins|Anti-Bacte...",NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,Gramcidin D is a heterogeneous mixture of thre...
33,DB00035,Desmopressin,small molecule,approved,H01BA02,Vasopressin V2 receptor|Vasopressin V1a recept...,"Agents that produce hypertension|Amino Acids, ...",NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,"Desmopressin (dDAVP), a synthetic analogue of ..."


In [152]:
# write drugbank tsv
# path = os.path.join('data', 'drugbank.tsv')
path = 'D:/WorkSpace/master_thesis/Drug_Bank/result/drugbank.tsv'
drugbank_df.to_csv(path, sep='\t', index=False)

# write slim drugbank tsv
# path = os.path.join('data', 'drugbank-slim.tsv')
path = 'D:/WorkSpace/master_thesis/Drug_Bank/result/drugbank-slim.tsv'
drugbank_slim_df.to_csv(path, sep='\t', index=False)

# approved drug data
drugbank_approved_df = drugbank_df[
    drugbank_df.groups.map(lambda x: 'approved' in x)]
path = 'D:/WorkSpace/master_thesis/Drug_Bank/result/drugbank_approved.tsv'
drugbank_approved_df.to_csv(path, sep='\t', index=False)

# write drugbank pickle data
import pickle 

with open("D:/WorkSpace/master_thesis/Drug_Bank/result/drugbank.pkl", "wb") as fw:
    pickle.dump(drugbank_df, fw)


## Extract protein information

In [243]:
protein_rows = list()
for i, drug in enumerate(root):
    drugbank_id = drug.findtext(ns + "drugbank-id[@primary='true']")
    for category in ['target', 'enzyme', 'carrier', 'transporter']:
        proteins = drug.findall('{ns}{cat}s/{ns}{cat}'.format(ns=ns, cat=category))
        for protein in proteins:
            row = {'drugbank_id': drugbank_id, 'category': category}
            row['organism'] = protein.findtext('{}organism'.format(ns))
            row['known_action'] = protein.findtext('{}known-action'.format(ns))
            actions = protein.findall('{ns}actions/{ns}action'.format(ns=ns))
            row['actions'] = '|'.join(action.text for action in actions)
            uniprot_ids = [polypep.text for polypep in protein.findall(
                "{ns}polypeptide/{ns}external-identifiers/{ns}external-identifier[{ns}resource='UniProtKB']/{ns}identifier".format(ns=ns))]            
            if len(uniprot_ids) != 1:
                continue
            row['uniprot_id'] = uniprot_ids[0]
            ref_text = protein.findtext("{ns}references[@format='textile']".format(ns=ns))
            pmids = re.findall(r'pubmed/([0-9]+)', ref_text)
            row['pubmed_ids'] = '|'.join(pmids)
            protein_rows.append(row)

protein_df = pandas.DataFrame.from_dict(protein_rows)

In [239]:
# Read our uniprot to entrez_gene mapping
response = requests.get('http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz', stream=True)
text = io.TextIOWrapper(gzip.GzipFile(fileobj=response.raw))
uniprot_df = pandas.read_table(text, engine='python')
uniprot_df.rename(columns={'uniprot': 'uniprot_id', 'GeneID': 'entrez_gene_id'}, inplace=True)

# merge uniprot mapping with protein_df
entrez_df = protein_df.merge(uniprot_df, how='inner')

In [240]:
columns = ['drugbank_id', 'category', 'uniprot_id', 'entrez_gene_id', 'organism',
           'known_action', 'actions', 'pubmed_ids']
entrez_df = entrez_df[columns]

In [241]:
path = os.path.join('data', 'proteins.tsv')
entrez_df.to_csv(path, sep='\t', index=False)

In [251]:
# Number of unique genes with an interaction
len(set(entrez_df.entrez_gene_id))

3757

In [252]:
# Number of unique drugs  with an interaction
len(set(entrez_df.drugbank_id))

5878